In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook93c5abe0c4/__results__.html
/kaggle/input/notebook93c5abe0c4/__notebook__.ipynb
/kaggle/input/notebook93c5abe0c4/__output__.json
/kaggle/input/notebook93c5abe0c4/custom.css
/kaggle/input/notebook93c5abe0c4/stack/train.csv
/kaggle/input/notebook93c5abe0c4/stack/test.csv
/kaggle/input/ccf-data/ccf_data/test_1.csv
/kaggle/input/ccf-data/ccf_data/train_all.csv
/kaggle/input/ccf-data/ccf_data/train_2.csv
/kaggle/input/ccf-data/ccf_data/test_2.csv
/kaggle/input/ccf-data/ccf_data/w2v/1_total_fee.csv
/kaggle/input/ccf-data/ccf_data/w2v/4_total_fee.csv
/kaggle/input/ccf-data/ccf_data/w2v/2_total_fee.csv
/kaggle/input/ccf-data/ccf_data/w2v/3_total_fee.csv


In [2]:
import os
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

path = "/kaggle/input/"

data_path = path + "ccf-data/ccf_data/"
w2v_path = path + "ccf-data/ccf_data/w2v/"
stack_path = path + "notebook93c5abe0c4/stack/"

# 原始数据
train = pd.read_csv(data_path + "train_2.csv")
test = pd.read_csv(data_path + "test_2.csv")
print("合并stacking前...")
print(len(train), len(test))

# stacking特征
train_stacking = pd.read_csv(stack_path + "train.csv")
test_stacking = pd.read_csv(stack_path + "test.csv")

# 合并
train = train.merge(train_stacking, "left", "user_id")
test = test.merge(test_stacking, "left", "user_id")
print("合并stacking后...")
print(len(train), len(test))

train_first = pd.read_csv(data_path + "train_all.csv")

# data_type==0为复赛训练集和测试集
# data_type==1为初赛训练集

train["data_type"] = 0
test["data_type"] = 0
train_first["data_type"] = 1

# 拼接
data = pd.concat([train, test], ignore_index=True).fillna(0) # 将测试集中current_service填充0
train_first = train_first.fillna(0)

data["label"] = data.current_service.astype(int)
data = data.replace("\\N", 0)
train_first = train_first.replace("\\N", 0)
data["gender"] = data.gender.astype(int)
train_first["gender"] = train_first.gender.astype(int)

# 初赛训练集没有service_type==3, 因此将复赛中少量的service_type==3的变更为4
data.loc[data["service_type"] == 3, "service_type"] = 4

# 原始类别特征
origin_cate_feature = ['service_type', 'complaint_level', 'contract_type', 'gender', 'is_mix_service',
                       'is_promise_low_consume',
                       'many_over_bill', 'net_service']

# 原始数值特征
origin_num_feature = ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee',
                      'age', 'contract_time',
                      'former_complaint_fee', 'former_complaint_num',
                      'last_month_traffic', 'local_caller_time', 'local_trafffic_month', 'month_traffic',
                      'online_time', 'pay_num', 'pay_times', 'service1_caller_time', 'service2_caller_time']

# 转换为浮点数
for i in origin_num_feature:
    data[i] = data[i].astype(float)
    train_first[i] = train_first[i].astype(float)

# 数据集和词向量特征合并
w2v_features = []
for col in ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']:
    df = pd.read_csv(w2v_path + "/" + col + ".csv")
    df = df.drop_duplicates([col]) # 去除重复值
    fs = list(df)
    fs.remove(col) # 移除原始特征1_total_fee
    w2v_features += fs
    print(len(data))
    data = pd.merge(data, df, on=col, how="left") # 合并
    print(len(data))
print("w2v_features:{}".format(w2v_features))

count_feature_list = []

# 新增count特征
def feature_count(data, features=[]):
    if len(set(features)) != len(features):
        print("equal feature !!!")
        return data
    new_feature = "count"
    for i in features:
        new_feature += "_" + i.replace("add_", "")
    # temp包含features和size特征
    temp = data.groupby(features).size().reset_index().rename(columns={0:new_feature})
    # 以features合并temp
    data = data.merge(temp, "left", on=features)
    count_feature_list.append(new_feature)
    # 包含service_type时，合并train_first的count特征
    if "service_type" in features:
        temp_2 = train_first.groupby(features).size().reset_index().rename(columns={0:"train_" + new_feature})
        data = data.merge(temp_2, "left", on=features)
        count_feature_list.append("train_" + new_feature)
    return data

# 四个月缴费金额
data = feature_count(data, ['1_total_fee'])
data = feature_count(data, ['2_total_fee'])
data = feature_count(data, ['3_total_fee'])
data = feature_count(data, ['4_total_fee'])

# 历史执行补救费用交费金额
data = feature_count(data, ['former_complaint_fee'])

# 交费金额
data = feature_count(data, ['pay_num'])
# 合约时长
data = feature_count(data, ['contract_time'])
# 上月结转流量
data = feature_count(data, ['last_month_traffic'])
# 在网时长
data = feature_count(data, ['online_time'])

# 套餐类型和合约类型与其他缴费相关
for i in ['service_type', 'contract_type']:
    data = feature_count(data, [i, '1_total_fee'])
    data = feature_count(data, [i, '2_total_fee'])
    data = feature_count(data, [i, '3_total_fee'])
    data = feature_count(data, [i, '4_total_fee'])
    # 历史执行补救费用交费金额	
    data = feature_count(data, [i, 'former_complaint_fee'])
    
    data = feature_count(data, [i, 'pay_num'])
    data = feature_count(data, [i, 'contract_time'])
    data = feature_count(data, [i, 'last_month_traffic'])
    data = feature_count(data, [i, 'online_time'])
    
# 插值特征
diff_feature_list = ['diff_total_fee_1', 'diff_total_fee_2', 'diff_total_fee_3', 'last_month_traffic_rest',
                     'rest_traffic_ratio',
                     'total_fee_mean', 'total_fee_max', 'total_fee_min', 'total_caller_time', 'service2_caller_ratio',
                     'local_caller_ratio',
                     'total_month_traffic', 'month_traffic_ratio', 'last_month_traffic_ratio', 'pay_num_1_total_fee',
                     '1_total_fee_call_fee', '1_total_fee_call2_fee', '1_total_fee_trfc_fee']

# 两个月之间的交费金额差值
data['diff_total_fee_1'] = data['1_total_fee'] - data['2_total_fee']
data['diff_total_fee_2'] = data['2_total_fee'] - data['3_total_fee']
data['diff_total_fee_3'] = data['3_total_fee'] - data['4_total_fee']

# 交费金额和当前总出账金额_月的差值
data['pay_num_1_total_fee'] = data['pay_num'] - data['1_total_fee']

# 流量差值
data['last_month_traffic_rest'] = data['month_traffic'] - data['last_month_traffic']
data['last_month_traffic_rest'][data['last_month_traffic_rest'] < 0] = 0  # 流量差值小于0的变为0
data['rest_traffic_ratio'] = (data['last_month_traffic_rest'] * 15 / 1024) / data['1_total_fee']

total_fee = []
for i in range(1, 5):
    total_fee.append(str(i) + '_total_fee')
    
# 4个月出账金额均值，最大值，最小值
data['total_fee_mean'] = data[total_fee].mean(1)
data['total_fee_max'] = data[total_fee].max(1)
data['total_fee_min'] = data[total_fee].min(1)

# 通话时长
data['total_caller_time'] = data['service2_caller_time'] + data['service1_caller_time']
data['service2_caller_ratio'] = data['service2_caller_time'] / data['total_caller_time']
data['local_caller_ratio'] = data['local_caller_time'] / data['total_caller_time']

# 累计流量
data['total_month_traffic'] = data['local_trafffic_month'] + data['month_traffic']
data['month_traffic_ratio'] = data['month_traffic'] / data['total_month_traffic']
data['last_month_traffic_ratio'] = data['last_month_traffic'] / data['total_month_traffic']

# 出账费用和套餐外通话时长费用
data['1_total_fee_call_fee'] = data['1_total_fee'] - data['service1_caller_time'] * 0.15
data['1_total_fee_call2_fee'] = data['1_total_fee'] - data['service2_caller_time'] * 0.15
data['1_total_fee_trfc_fee'] = data['1_total_fee'] - (
        data['month_traffic'] - 2 * data['last_month_traffic']) * 0.3

# 套餐1，出账金额差值
data.loc[data.service_type == 1, '1_total_fee_trfc_fee'] = None

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


合并stacking前...
374655 160566
合并stacking后...
374655 160566


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (4,5,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


535221
535221
535221
535221
535221
535221
535221
535221
w2v_features:['1_total_feeW0', '1_total_feeW1', '1_total_feeW2', '1_total_feeW3', '1_total_feeW4', '1_total_feeW5', '1_total_feeW6', '1_total_feeW7', '1_total_feeW8', '1_total_feeW9', '2_total_feeW0', '2_total_feeW1', '2_total_feeW2', '2_total_feeW3', '2_total_feeW4', '2_total_feeW5', '2_total_feeW6', '2_total_feeW7', '2_total_feeW8', '2_total_feeW9', '3_total_feeW0', '3_total_feeW1', '3_total_feeW2', '3_total_feeW3', '3_total_feeW4', '3_total_feeW5', '3_total_feeW6', '3_total_feeW7', '3_total_feeW8', '3_total_feeW9', '4_total_feeW0', '4_total_feeW1', '4_total_feeW2', '4_total_feeW3', '4_total_feeW4', '4_total_feeW5', '4_total_feeW6', '4_total_feeW7', '4_total_feeW8', '4_total_feeW9']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
# 类别特征
cate_feature = origin_cate_feature
# 数值特征
num_feature = origin_num_feature + count_feature_list + diff_feature_list + w2v_features + ['stacking_' + str(i) for i
                                                                                            in range(11)]

# 变更数据格式
for i in cate_feature:
    data[i] = data[i].astype('category')
for i in num_feature:
    data[i] = data[i].astype(float)

# 所有特征
feature = cate_feature + num_feature
print(len(feature), feature)

130 ['service_type', 'complaint_level', 'contract_type', 'gender', 'is_mix_service', 'is_promise_low_consume', 'many_over_bill', 'net_service', '1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee', 'age', 'contract_time', 'former_complaint_fee', 'former_complaint_num', 'last_month_traffic', 'local_caller_time', 'local_trafffic_month', 'month_traffic', 'online_time', 'pay_num', 'pay_times', 'service1_caller_time', 'service2_caller_time', 'count_1_total_fee', 'count_2_total_fee', 'count_3_total_fee', 'count_4_total_fee', 'count_former_complaint_fee', 'count_pay_num', 'count_contract_time', 'count_last_month_traffic', 'count_online_time', 'count_service_type_1_total_fee', 'train_count_service_type_1_total_fee', 'count_service_type_2_total_fee', 'train_count_service_type_2_total_fee', 'count_service_type_3_total_fee', 'train_count_service_type_3_total_fee', 'count_service_type_4_total_fee', 'train_count_service_type_4_total_fee', 'count_service_type_former_complaint_fee', 'train_coun

In [4]:
# 训练
lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt", num_leaves=152, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1000, objective='multiclass', class_weight="balanced",
    subsample=0.9, colsample_bytree=0.5, subsample_freq=1,
    learning_rate=0.03, random_state=2018, n_jobs=10
)
# 复赛训练数据
lgb_model.fit(data[data.label != 0][feature],data[data.label != 0].label, categorical_feature=cate_feature)

result_type1 = pd.DataFrame()

# 复赛测试集中service_type==1时的user_id和predict
result_type1["user_id"] = data[(data.label == 0) & (data.service_type == 1)]["user_id"]
result_type1["predict"] = lgb_model.predict(data[(data.label == 0) & (data.service_type == 1)][feature])
print(len(result_type1))

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


106801


In [5]:
def f1_marco(pred, labels):
    labels = np.argmax(labels.reshape(8, -1), axis=0)
    score = f1_score(y_true=labels, y_pred=preds, average="macro")

In [6]:
# 复赛训练集
X = data[(data.label != 0) & (data.label != 999999)][feature].reset_index(drop=True)
y = data[(data.label != 0) & (data.label != 999999)].label.reset_index(drop=True)

label2current_service = dict(
    zip(range(0, len(set(y))), sorted(list(set(y))))
)
current_service2label = dict(
    zip(sorted(list(set(y))), range(0, len(set(y))))
)

cv_pred = []
#5折交叉验证
skf = StratifiedKFold(n_splits=5, random_state=20181, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(index)
    lgb_model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=120, reg_alpha=0, reg_lambda=0.,
        max_depth=-1, n_estimators=800, objective='multiclass', class_weight='balanced',
        subsample=0.9, colsample_bytree=0.5, subsample_freq=1,
        learning_rate=0.03, random_state=2018 + index, n_jobs=10, metric="None", importance_type='gain'
    )
    # 训练和测试数据
    train_x, test_x, train_y, test_y = X.loc[train_index], X.loc[test_index], y.loc[train_index], y.loc[test_index]
    
    # service_type == 4
    train_x = train_x[train_x.service_type == 4]
    train_y = train_y[(train_x.service_type == 4).index]
    test_x = test_x[test_x.service_type == 4]
    test_y = test_y[(test_x.service_type == 4).index]
    print(test_y.unique())
    
    eval_set = [(test_x, test_y)]
    lgb_model.fit(train_x, train_y, categorical_feature=cate_feature)
    # 每次都要预测一下y_test
    y_test = lgb_model.predict(data[(data.label == 0) & (data.service_type != 1)][feature])
    y_test = pd.Series(y_test).map(current_service2label)
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))  # 沿着水平方向拼接

# cv_pred每一列都是一组预测值，bincount计数得到数量最多的预测值
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line))) # 计数
result = pd.DataFrame()
result['user_id'] = data[(data.label == 0) & (data.service_type != 1)]['user_id']
result['predict'] = submit
result['predict'] = result['predict'].map(label2current_service)
result.loc[result['user_id'] == '4VNcD6kE0sjnAvFX', 'predict'] = 999999
result = result.append(result_type1)

print(len(result), result.predict.value_counts())
print(result.sort_values('user_id').head())

output_path = "/kaggle/working"
result[['user_id', 'predict']].to_csv(
    output_path + '/sub1.csv', index=False)

0
[89950166 99999828 99999825 89950167 99999830 89950168 99999826 99999827]


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


1
[99999830 99999828 89950168 89950167 99999826 89950166 99999825 99999827]


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


2
[99999826 89950166 99999828 89950167 99999830 89950168 99999827 99999825]


...

3
[99999827 99999828 89950166 99999826 89950167 99999830 99999825 89950168]


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


4
[89950167 99999827 99999830 89950168 99999826 99999828 89950166 99999825]


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


160566 90063345    93141
89950167    16658
89950166    12352
90109916     7996
99999828     7321
90155946     5664
99999826     4162
99999830     3866
99999827     3664
89950168     3066
99999825     2675
999999          1
Name: predict, dtype: int64
                 user_id   predict
374655  013mNJ4xUaAzRLG9  90063345
374656  013oEGVd7HIWtvPZ  90063345
374657  0156TRH7IzXNh8pj  99999830
374658  019EdSRKXsYDvGNM  89950166
374659  019SyqXtp24FW5gx  89950167
